# Задание 1
Для датафрейма `log` создайте столбец `source_type` по правилам:

1. Если источник `traffic_source` равен **Yandex** или **Google** → `source_type` = `'organic'`
2. Для источников **paid** и **email** из России → `'ad'`
3. Для источников **paid** и **email` не из России → `'other'`
4. Остальные варианты берём из `traffic_source` без изменений.

In [ ]:
import pandas as pd

df = pd.read_csv('data/advanced_pandas_data/visit_log.csv',
                 header=None,
                 names=['timestamp', 'visit_id', 'url', 'region', 'user_id', 'traffic_source'])

def create_source_type(row):
    traffic_source = row['traffic_source']
    region = row['region']

    if traffic_source in ['yandex', 'google']:
        return 'organic'
    elif traffic_source in ['paid', 'email'] and region == 'Russia':
        return 'ad'
    elif traffic_source in ['paid', 'email'] and region != 'Russia':
        return 'other'
    else:
        return traffic_source
print(df.columns)
df['source_type'] = df.apply(create_source_type, axis=1, sep=',')

output_file = 'data/advanced_pandas_data/visit_log_with_source_type.csv'
df.to_csv(output_file, index=False)

print(f"Результат сохранен в файл: {output_file}")

# Задание 2
В файле `URLs.txt` содержатся URL страниц новостного сайта.
Нужно отфильтровать его по адресам **страниц с текстами новостей**.

**Шаблон страницы новостей:**
Внутри URL есть конструкция: `/`, затем 8 цифр, затем дефис.

**Действия:**
1. Прочитайте содержимое файла в датафрейм.
2. Отфильтруйте страницы с текстом новостей, используя метод `str.contains` и **регулярное выражение** в соответствии с заданным шаблоном.

In [4]:
import pandas as pd
import re

df = pd.read_csv('data/advanced_pandas_data/URLs.txt')

url_column = 'url' if 'url' in df.columns else df.columns[0]

pattern = r'/\d{8}-'
news_df = df[df[url_column].str.contains(re.compile(pattern), na=False)]

for i, url in enumerate(news_df[url_column].head(20), 1):
    print(f"{i:2}. {url}")

output_file = 'filtered_news_urls.csv'
news_df.to_csv(output_file, index=False)

print(f"\nРезультат сохранен в файл: {output_file}")

Используем колонку: 'url'
 1. /politics/36188461-s-marta-zhizn-rossiyan-suschestvenno-izmenitsya-iz-za-novyh-zakonov/
 2. /world/36007585-tramp-pridumal-kak-reshit-ukrainskiy-vopros/
 3. /science/36157853-nasa-sobiraet-ekstrennuyu-press-konferentsiyu-na-temu-vnezemnoy-zhizni/
 4. /video/36001498-poyavilis-pervye-podrobnosti-gibeli-natali-melamed/
 5. /world/36007585-tramp-pridumal-kak-reshit-ukrainskiy-vopros/?smi2=1
 6. /middleeast/36131117-divizion-s-400-ne-zametil-ataki-f-35-pod-damaskom/
 7. /economics/36065674-rossiyane-vozmutilis-minimalnymi-zarplatami-v-stranah-es/
 8. /politics/36118047-otets-24-detey-potreboval-izvineniy-ot-soloveva/?smi2=1
 9. /travel/36194479-v-sheremetevo-passazhiry-ustroili-bunt/
10. /politics/35638742-nesolidno-mariya-zaharova-sdelala-neozhidannoe-priznanie/
11. /video/36012692-tramp-otkrestilsya-ot-rossii-i-putina/
12. /starlife/36174817-guzeeva-rasstalas-s-muzhem/
13. /health/36149308-predlozheno-prostoe-sredstvo-ot-insulta/
14. /science/36139723-set-vz

# Задание 3
Используйте файл с оценками фильмов `ml-latest-small/ratings.csv`.

**Задача:**
Посчитайте **среднее время жизни пользователей**, которые выставили более 100 оценок.

**Определение времени жизни:**
Разница между максимальным и минимальным значениями столбца `timestamp` для данного `userId`.

In [1]:
import pandas as pd

# Загрузка и преобразование данных
df = pd.read_csv('data/advanced_pandas_data/ml-latest-small/ratings.csv',
                 header=None,
                 names=['userId', 'movieId', 'rating', 'timestamp'])
df['timestamp'] = pd.to_numeric(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp'])

# Расчет
user_stats = df.groupby('userId')['timestamp'].agg(['min', 'max', 'count'])
user_stats['lifetime'] = user_stats['max'] - user_stats['min']
active_users = user_stats[user_stats['count'] > 100]
avg_lifetime = active_users['lifetime'].mean() if len(active_users) > 0 else 0

print(f"Среднее время жизни: {avg_lifetime:.0f} сек ({avg_lifetime/(24*60*60):.1f} дней)")

Среднее время жизни: 40080507 сек (463.9 дней)


# Задание 4
Дана статистика услуг перевозок клиентов компании по типам (см. файл `Python_13_join.ipynb` в разделе «Материалы для лекции “Продвинутый pandas”»).

**Нужно сформировать две таблицы:**

1. Таблицу с тремя типами выручки для каждого `client_id` **без указания адреса клиента**.
2. Аналогичную таблицу по типам выручки **с указанием адреса клиента**.

In [2]:
import pandas as pd

rzd = pd.DataFrame({
    'client_id': [111, 112, 113, 114, 115],
    'rzd_revenue': [1093, 2810, 10283, 5774, 981]
})

auto = pd.DataFrame({
    'client_id': [113, 114, 115, 116, 117],
    'auto_revenue': [57483, 83, 912, 4834, 98]
})

air = pd.DataFrame({
    'client_id': [115, 116, 117, 118],
    'air_revenue': [81, 4, 13, 173]
})

client_base = pd.DataFrame({
    'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
    'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
})

table_without_addresses = (client_base[['client_id']]
                           .merge(rzd, how='left')
                           .merge(auto, how='left')
                           .merge(air, how='left')
                           .fillna(0))

table_with_addresses = table_without_addresses.merge(client_base, on='client_id')
table_with_addresses = table_with_addresses[['client_id', 'address', 'rzd_revenue', 'auto_revenue', 'air_revenue']]

print("Таблица без адресов:")
print(table_without_addresses)
print("\nТаблица с адресами:")
print(table_with_addresses)

Таблица без адресов:
   client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           0.0          0.0
1        112       2810.0           0.0          0.0
2        113      10283.0       57483.0          0.0
3        114       5774.0          83.0          0.0
4        115        981.0         912.0         81.0
5        116          0.0        4834.0          4.0
6        117          0.0          98.0         13.0
7        118          0.0           0.0        173.0

Таблица с адресами:
   client_id          address  rzd_revenue  auto_revenue  air_revenue
0        111  Комсомольская 4       1093.0           0.0          0.0
1        112   Энтузиастов 8а       2810.0           0.0          0.0
2        113  Левобережная 1а      10283.0       57483.0          0.0
3        114          Мира 14       5774.0          83.0          0.0
4        115        ЗЖБИиДК 1        981.0         912.0         81.0
5        116    Строителей 18          0.0        4834.0     